In [1]:
!pip install flask flask-ngrok
!pip install git+https://github.com/huggingface/transformers -U
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install trl
!pip install git+https://github.com/huggingface/peft.git


  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-um389hkt
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-um389hkt
  Resolved https://github.com/huggingface/transformers to commit ac946aac257cadfa8264fa4a284cd0ea1061c5b5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.43.0.dev0-py3-none-any.whl size=9354096 sha256=e1e0efa8af95631c744fd8765b027b3c2130e8e98b460c43b09f5892a91694c0
  Stored in directory: /tmp/pip-ephem-wheel-cache-bdal1pdw/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import notebook_login
notebook_login(write_permission=True)

In [ ]:
!pip install pyngrok

In [ ]:
import torch
from flask import Flask, request, jsonify
from pyngrok import ngrok
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

app = Flask(__name__)

# Load pre-trained model and tokenizer
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-7b-it").to("cuda")

def generate_response(user_input):
    input_ids = tokenizer(user_input, return_tensors="pt").to("cuda")
    outputs = model.generate(**input_ids, max_length=1024, do_sample=True, top_k=10, top_p=0.9)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response



In [ ]:
# Define the HTML for the chat interface
html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Gemma-7b Chatbot</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 0; padding: 0; display: flex; flex-direction: column; height: 100vh; }
        #chatbox { flex: 1; padding: 10px; overflow-y: auto; border: 1px solid #ccc; }
        #input-box { display: flex; border-top: 1px solid #ccc; }
        #input-box input { flex: 1; padding: 10px; border: none; outline: none; }
        #input-box button { padding: 10px; border: none; cursor: pointer; background-color: #007BFF; color: white; }
        .message { margin: 10px 0; }
        .user { text-align: right; }
        .bot { text-align: left; }
    </style>
</head>
<body>
    <div id="chatbox"></div>
    <div id="input-box">
        <input type="text" id="user_input" placeholder="Type your message here..." />
        <button onclick="sendMessage()">Send</button>
    </div>

    <script>
        async function sendMessage() {
            const userInput = document.getElementById("user_input").value;
            if (userInput.trim() === "") return;

            addMessage(userInput, "user");
            document.getElementById("user_input").value = "";

            const response = await fetch("/chat", {
                method: "POST",
                headers: { "Content-Type": "application/json" },
                body: JSON.stringify({ user_input: userInput })
            });

            const data = await response.json();
            addMessage(data.response, "bot");
        }

        function addMessage(text, sender) {
            const messageDiv = document.createElement("div");
            messageDiv.classList.add("message", sender);
            messageDiv.textContent = text;
            document.getElementById("chatbox").appendChild(messageDiv);
            document.getElementById("chatbox").scrollTop = document.getElementById("chatbox").scrollHeight;
        }
    </script>
</body>
</html>
"""


In [ ]:
@app.route("/")
def index():
    return html_content

@app.route("/chat", methods=["POST"])
def chat():
    user_input = request.json["user_input"]
    model_response = generate_response(user_input)
    return jsonify({"response": model_response})

if __name__ == "__main__":
   public_url = ngrok.connect(5000).public_url
   print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:5000/\"".format(public_url))
app.run()